In [1]:
import json
import pandas as pd

word_list = []
with open(r"word_list\spanish\es_wordlist.json") as file:
    json_data = json.load(file)
    for d in json_data.items():
        word_list.append([d[0], d[1]])

In [2]:
df = pd.DataFrame(word_list)
df.columns = ["word", "count"]
df["word"] = df["word"].apply(lambda x: str(x).lower())
agg_functions = {"word": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"])
df.head()

,word,count,word_len
0,0,540301,1
1,00,28630,2
2,000,66843,3
3,0000,4062,4
4,00000,29,5


In [3]:
df.dtypes

word        object
count        int64
word_len     int64
dtype: object

In [4]:
df.describe()

,count,word_len
count,2.672628e+06,2.672628e+06
mean,3.857258e+02,9.975893e+00
std,6.993461e+04,6.187905e+00
min,1.000000e+00,1.000000e+00
25%,1.000000e+00,6.000000e+00
50%,2.000000e+00,8.000000e+00
75%,4.000000e+00,1.100000e+01
max,7.534259e+07,1.200000e+02


In [17]:
df[(df["word_len"] == 5) & (df["count"] > 2e3)].describe()

,count,word_len
count,2.450000e+03,2450.0
mean,3.530331e+04,5.0
std,1.120258e+05,0.0
min,2.001000e+03,5.0
25%,3.410750e+03,5.0
50%,7.131500e+03,5.0
75%,2.039400e+04,5.0
max,2.031992e+06,5.0


In [6]:
df.query("'libro' in word")

,word,count,word_len
1650975,libro,62042,5


In [ ]:
configs = [
    {"len": 5, "guess_min_counts": 1e2, "answer_min_counts": 2e3},
    {"len": 6, "guess_min_counts": 2e5, "answer_min_counts": 2e6},
    {"len": 7, "guess_min_counts": 2e5, "answer_min_counts": 2e6},
]


for config in configs:
    l = config["len"]

    with open(rf"vocabs\es-len{l}-guess.json", "w") as f:
        count = config["guess_min_counts"]
        df_filtered = df[(df["word_len"] == l) & (df["count"] >= count)]
        data = df_filtered["word"].tolist()
        json.dump(data, f)

    with open(rf"vocabs\es-len{l}-answer.json", "w") as f:
        count = config["answer_min_counts"]
        df_filtered = df[(df["word_len"] == l) & (df["count"] >= count)]
        data = df_filtered["word"].tolist()
        json.dump(data, f)